In [10]:
# set working directory :
import os
pwd = os.getcwd() + "/../"
os.chdir(pwd)


In [11]:
# load env
from dotenv import load_dotenv
import os 
load_dotenv("../.env")

from tickapp.clients.signal_client import SignalClient

In [12]:
# init client
client = SignalClient(phone_number=os.getenv("SIGNAL_PHONE_NUMBER"))

INFO:tickapp.clients.signal_client:✅ Signal Client initialisé pour +33695071416


In [13]:
msgs = client.receive(number_of_messages=100)

DEBUG:tickapp.clients.signal_client:🔧 Commande: signal-cli -a +33695071416 -o json -a +33695071416 receive --max-messages 100 --send-read-receipts


In [14]:
msgs_p = client._parse_message(msgs)
msgs_p

[Message(sender=Contact(number='', name='Carmelo Mileto', uuid='6aef6f35-7fcd-44c3-a6a4-e4c69f43535c'), timestamp=datetime.datetime(2025, 11, 30, 11, 46, 8, 283000), text='Carmelo', attachments=[Attachment(id='2ltlwohfVpAMoERvUG0j.jpg', content_type='image/jpeg', filename='signal-2025-11-25-231507.jpg', size=166675, upload_timestamp_ms=1764499568309, path=None)], group=Group(id='mPN0DQcs5cTJ+UMmJb9IDTGQMzCtPHP68fWQdZw/Uag=', name='Unknown'), is_group_message=True, account='')]

In [ ]:
from pydoc import cli


phone_number=client.phone_number
messages=msgs_p

for message in messages:
    for attachment in message.attachments:
        args = ["-a", phone_number, "getAttachment", "--id", attachment.id, "--group", message.group.id]
        print(args)
        client._run_command(args)
        output_path = client._get_signal_cli_attachments_dir() / attachment.id
        attachment.path = output_path



DEBUG:tickapp.clients.signal_client:🔧 Commande: signal-cli -a +33695071416 -a +33695071416 getAttachment --id 2ltlwohfVpAMoERvUG0j.jpg --group mPN0DQcs5cTJ+UMmJb9IDTGQMzCtPHP68fWQdZw/Uag=


['-a', '+33695071416', 'getAttachment', '--id', '2ltlwohfVpAMoERvUG0j.jpg', '--group', 'mPN0DQcs5cTJ+UMmJb9IDTGQMzCtPHP68fWQdZw/Uag=']


In [ ]:
# download attachment
msgs_dw = client.download_attachment(
    phone_number=client.phone_number, 
    messages=msgs_p
)
msgs_dw

[]

In [ ]:
msgs_dw[0].attachments[0].path

IndexError: list index out of range

In [ ]:
msgs_dw

[Message(sender=Contact(number='', name='Carmelo Mileto', uuid='6aef6f35-7fcd-44c3-a6a4-e4c69f43535c'), timestamp=datetime.datetime(2025, 11, 30, 11, 16, 50, 542000), text='C', attachments=[Attachment(id='2ltlwohfVpAMoERvUG0j.jpg', content_type='image/jpeg', filename='signal-2025-11-25-231507.jpg', size=166675, upload_timestamp_ms=1764497810582, path=PosixPath('/Users/carmelomileto/.local/share/signal-cli/attachments/2ltlwohfVpAMoERvUG0j.jpg'))], group=Group(id='mPN0DQcs5cTJ+UMmJb9IDTGQMzCtPHP68fWQdZw/Uag=', name='Unknown'), is_group_message=True, account='')]

In [ ]:
from tickapp.clients.database_client import DatabaseClient

In [ ]:
db_client = DatabaseClient(port=5434)
message_id, attachment_ids = db_client.insert_signal_message(msgs_dw[0])

✅ Message Signal inséré : message_id=1, 1 attachments
